<a href="https://colab.research.google.com/github/milver/Experiments/blob/main/TestBiomassStreaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing ABGB dataset streaming from HuggingFace

## Install necessary libraries

In [1]:
!pip install datasets leafmap geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.9/182.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

## Import necessary libraries

In [5]:
from datasets import load_dataset
import leafmap
import geopandas as gpd
from shapely.geometry import Point
import ipywidgets as widgets

## Functions for stream map

In [10]:
# Function to filter dataset based on map bounds
def filter_dataset(bounds, dataset):
    filtered_data = {'latitude': [], 'longitude': [], 'biomass': []}
    for sample in dataset:
        lat = sample['latitude']
        lon = sample['longitude']
        if bounds['south'] <= lat <= bounds['north'] and bounds['west'] <= lon <= bounds['east']:
            filtered_data['latitude'].append(lat)
            filtered_data['longitude'].append(lon)
            filtered_data['biomass'].append(sample['biomass'])
    return filtered_data

# Function to update the map based on current bounds
def update_map(m, dataset):
    bounds = m.get_bounds()
    filtered_data = filter_dataset(bounds, dataset)
    geometry = [Point(xy) for xy in zip(filtered_data['longitude'], filtered_data['latitude'])]
    gdf = gpd.GeoDataFrame({'biomass': filtered_data['biomass']}, geometry=geometry)
    m.add_gdf(gdf, layer_name='Biomass Data', zoom_to_layer=False)

## Load the dataset

In [13]:
# Load the dataset
dataset = load_dataset("prs-eth/AGBD_15", streaming=True)["train"]

Resolving data files:   0%|          | 0/461 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/125 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/153 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/461 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/125 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/153 [00:00<?, ?it/s]

## Initialize and Map

In [ ]:
# Initialize a Leafmap map
m = leafmap.Map(center=(39.0170445, -77.5782395), zoom=13)

# Create a button to update the map
update_button = widgets.Button(description="Update Map")
update_button.on_click(lambda x: update_map(m, dataset))

# Display the button and map
display(update_button)
m